In [1]:
from HATN import *
import numpy as np
import pandas as pd
import os


Using TensorFlow backend.


In [2]:
with open(os.path.join(os.getcwd(), '../Data/spam.csv'), 'r',  encoding='latin-1') as f:
    data = pd.read_csv(f)

In [3]:
data.drop(labels=['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], axis=1, inplace=True)

In [4]:
# Ham: 0, Spam: 1
labels = data['v1'].astype('category').cat.codes

In [5]:
weight_path = os.path.join(os.getcwd(), '../Data/glove.840B.300d.txt')

In [6]:
(x_train, y_train), (x_val, y_val), word_index, embedding_matrix = text_processing(data['v2'].values, labels, weights_path=weight_path)

Total 8919 unique tokens.
Shape of data tensor: (5572, 30, 100)
Shape of label tensor: (5572,)
Number of spams in the training set is 611/4458
Number of spams in the training set is 136/1114
Total 2195885 word vectors.


In [13]:
def HATN(num_words, trainable=True, embedding_matrix=None):
    if not trainable:
        embedding_layer = Embedding(num_words,
                                    EMBEDDING_DIM,
                                    weights=[embedding_matrix],
                                    input_length=MAX_SENT_LENGTH,
                                    trainable=False,
                                    mask_zero=False)

    else:
        embedding_layer = Embedding(num_words,
                                    EMBEDDING_DIM,
                                    embeddings_initializer='uniform',
                                    input_length=MAX_SENT_LENGTH,
                                    trainable=True,
                                    mask_zero=True)

    sentence_input = Input(shape=(MAX_SENT_LENGTH,), dtype='int32')
    embedded_sequences = embedding_layer(sentence_input)
    l_lstm = Bidirectional(GRU(100, return_sequences=True))(embedded_sequences)
    l_att = AttLayer(100)(l_lstm)
    sentEncoder = Model(sentence_input, l_att)

    review_input = Input(shape=(MAX_SENTS, MAX_SENT_LENGTH), dtype='int32')
    review_encoder = TimeDistributed(sentEncoder)(review_input)
    l_lstm_sent = Bidirectional(GRU(100, return_sequences=True))(review_encoder)
    l_att_sent = AttLayer(100)(l_lstm_sent)
    preds = Dense(1, activation='softmax')(l_att_sent)
    model = Model(review_input, preds)

    return model

model = HATN(len(word_index) + 1, trainable=False, embedding_matrix = embedding_matrix)

In [19]:
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

In [1]:
print("model fitting - Hierachical attention network")
model.fit(x_train, y_train, 
          validation_data=(x_val, y_val),
          epochs=10, 
          batch_size=50)

model fitting - Hierachical attention network


NameError: name 'model' is not defined